In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

## Organize Data

In [ ]:
BUFFER_SIZE=10000
BATCH_SIZE=32
IMG_SIZE=(32,32)

train_data = tf.keras.preprocessing.image_dataset_from_directory('/kaggle/input',
                                                                 validation_split=.9,
                                                                 subset='training',
                                                                 seed=123,
                                                                 image_size=IMG_SIZE,
                                                                 batch_size=BATCH_SIZE,
                                                                 shuffle=True)
val_data = tf.keras.preprocessing.image_dataset_from_directory('/kaggle/input',
                                                               validation_split=.9,
                                                               subset='validation',
                                                               seed=123,
                                                               image_size=IMG_SIZE,
                                                               batch_size=BATCH_SIZE)

#train_data = train_data.map(lambda img, lab: tf.image.resize(img, IMG_SIZE))
#val_data = val_data.map(lambda img, lab: tf.image.resize(img, IMG_SIZE))

#train_data = train_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
#val_data = val_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
image_dir = '/kaggle/input/gananime-lite/'
[os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]

In [ ]:
image_paths = '/kaggle/input/gananime-lite/out2/seed1000.png'

def load_and_decode_image(file_path):
    # Read the image from file
    image = tf.io.read_file(file_path)
    # Decode the image (adjust this according to your image format)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    # You may also want to normalize and resize the image here
    image = tf.image.resize(image, IMG_SIZE)
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return image

image_paths_tensor = tf.constant([image_paths])
dataset = tf.data.Dataset.from_tensor_slices(image_paths_tensor)
dataset = dataset.map(load_and_decode_image)
dataset = tf.data.Dataset.zip((dataset, tf.data.Dataset.from_tensor_slices([1])))
train_data = dataset.repeat(10000).batch(BATCH_SIZE)

### Observe first 10 training images

For the sake of computing resource limitations, we use only a single training image repeated 10,000 times. GANs can take many iterations to successfully learn how to construct new images from a high-dimensional vector space, so we reduce the complexity of the vector space by repeating the same image.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for img, lab in train_data.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        currimg = img[i]
        currimg = currimg*127.5+127.5
        plt.imshow(currimg.numpy().astype('uint8'))

## Build Models

### Generator model

In [ ]:
from tensorflow.keras import layers

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(4*4*256, use_bias=False, input_shape=(100,)))
    #model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((4, 4, 256)))
    assert model.output_shape == (None, 4, 4, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 8, 8, 128)
    #model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 16, 16, 128)
    #model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 32, 32, 128)
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2D(3, (3,3), activation='tanh', padding='same'))

    return model

### Random noise through untrained generator

In [ ]:
generator = make_generator_model()
noise = tf.random.normal([1,100])
gen_img = generator(noise, training=False)
plt.imshow(gen_img[0]*127.5)

### Discriminator model

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[32, 32, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

### Decision made by discriminator on previously generated image

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(gen_img)
print(decision)

## Model Training

### Create loss functions and setup training

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)

gen_opt = tf.keras.optimizers.Adam(1e-4)
disc_opt = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = '/kaggle/working/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=gen_opt,
                                 discriminator_optimizer=disc_opt,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
EPOCHS=10
noise_dim=100
num_examples=4

seed = tf.random.normal([num_examples, noise_dim])

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_imgs = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(gen_imgs, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        
    gen_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    gen_opt.apply_gradients(zip(gen_grads, generator.trainable_variables))
    disc_opt.apply_gradients(zip(disc_grads, discriminator.trainable_variables))
    
    

### Step through training process

In [ ]:
from tqdm import tqdm
from IPython import display

def train(dataset, epochs):
    num_batches = dataset.cardinality().numpy()
    
    for epoch in range(epochs):
        print("EPOCH ", epoch)
        
        for image_batch, labs in tqdm(dataset):
            train_step(image_batch)
            
        # Produce images for the GIF as you go
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                             epoch + 1,
                             seed)
            
        if (epoch+1) % 2 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
            
    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(2, 2))

    for i in range(predictions.shape[0]):
        plt.subplot(2, 2, i+1)
        plt.imshow(predictions[i, :, :, :] * 127.5 + 127.5)
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
train(train_data, EPOCHS)

In [ ]:
noise = tf.random.normal([1,100])
gen_img = generator(noise, training=False)
plt.imshow(gen_img[0,:,:,:])

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))